In [1]:
#%matplotlib qt

import time
import copy
import numpy as np
from skimage.filters import gaussian_filter

from tifffile import imread, imsave
import pickle

import matplotlib.pyplot as plt

from spimagine import volfig, volshow
from spimagine import EllipsoidMesh, Mesh

%reload_ext autoreload
%autoreload 2

from netsurface3d import NetSurf3d
from data4d import Data4d

# import os
# os.environ['PYOPENCL_COMPILER_OUTPUT'] = '1'

no scikit-tensor detected, so separable functions wont work
This is a python3 issue and can be resolved by:

git clone https://github.com/mnick/scikit-tensor.git
pip3 install scikit-tensor


# Load time-lapse data...

In [2]:
filenames = ['/Users/prakash/Desktop/092216_PH-mCherry-02_w2SD mCherry_t1.TIF'] #,
#             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t2.TIF',
#             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t3.TIF',
#             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t4.TIF',
#             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t5.TIF',
#             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t6.TIF',
#             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t7.TIF',
#             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t8.TIF',
#             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t9.TIF',
#             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t10.TIF',
#             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t11.TIF',
#             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t12.TIF']

In [3]:
data = Data4d( filenames, pixelsize=(1.,1.,5.), silent=False )
data.set_seg_params(K=30,max_delta_k=4)
objects = []
ecells = []

Dimensions of frame 0 :  (31, 512, 512)


Min radius and max radius at which to look for cells (around center points (seed points))

In [4]:
max_rs = (data.images[0].shape[2] / 8,
          data.images[0].shape[1] / 8,
          data.images[0].shape[0] / 2)

min_rs = (5,5,1)

## adding E cells

In [5]:
Ea = data.init_object("Ea")
objects.append(Ea)
ecells.append(Ea)

Ep = data.init_object("Ep")
objects.append(Ep)
ecells.append(Ep)

In [6]:
#data.add_object_at( Ea, min_rs, max_rs, frame=0, seed=(110,300,12), frame_to=11, seed_to=(105,310,16), segment_it=True )
data.add_object_at( Ea, min_rs, max_rs, frame=0, seed=(110,300,12), segment_it=True )
data.track(Ea, seed_frame=0, target_frames=range(1,len(data.images)), recenter_iterations=1)

Added appearance for "Ea" in frame 0 with seed coordinates [ 110.  300.   12.]
      Optimum energy:  95372.0
      Nodes in/out:  7366 7994


In [7]:
#data.add_object_at( Ep, min_rs, max_rs, frame=0, seed=(110,350,12), frame_to=11, seed_to=(140,360,16), segment_it=True )
data.add_object_at( Ep, min_rs, max_rs, frame=0, seed=(110,350,12), segment_it=True )
data.track(Ep, seed_frame=0, target_frames=range(1,len(data.images)), recenter_iterations=1)

Added appearance for "Ep" in frame 0 with seed coordinates [ 110.  350.   12.]
      Optimum energy:  88083.0
      Nodes in/out:  7008 8352


## adding direct neighbors

In [8]:
neighbors = []
for i in range(12):
    n = data.init_object("n"+str(i))
    objects.append(n)
    neighbors.append(n)

In [17]:
nseeds = [( 90,390,12),
          (150,410,12),
          (200,390,12),
          (190,315,12),
          (145,265,12),
          (110,230,12),
          ( 40,255,12),
          ( 45,300,18),
          ( 67,355,18),
          (110,275,24),
          (130,315,24),
          (125,360,24)]
for i in range(len(nseeds)):
    data.add_object_at( neighbors[i], min_rs, max_rs, frame=0, seed=nseeds[i], segment_it=True )
    data.track(neighbors[i], seed_frame=0, target_frames=range(1,len(data.images)), recenter_iterations=1)

Added appearance for "n0" in frame 0 with seed coordinates [  90.  390.   12.]
      Optimum energy:  76013.0
      Nodes in/out:  4767 10593
Added appearance for "n1" in frame 0 with seed coordinates [ 150.  410.   12.]
      Optimum energy:  76241.0
      Nodes in/out:  5712 9648
Added appearance for "n2" in frame 0 with seed coordinates [ 200.  390.   12.]
      Optimum energy:  76922.0
      Nodes in/out:  6544 8816
Added appearance for "n3" in frame 0 with seed coordinates [ 190.  315.   12.]
      Optimum energy:  88296.0
      Nodes in/out:  6602 8758
Added appearance for "n4" in frame 0 with seed coordinates [ 145.  265.   12.]
      Optimum energy:  89403.0
      Nodes in/out:  6512 8848
Added appearance for "n5" in frame 0 with seed coordinates [ 110.  230.   12.]
      Optimum energy:  91228.0
      Nodes in/out:  7294 8066
Added appearance for "n6" in frame 0 with seed coordinates [  40.  255.   12.]
      Optimum energy:  76758.0
      Nodes in/out:  6235 9125
Added appear

## adding others

In [10]:
others = []
for i in range(7):
    o = data.init_object("o"+str(i))
    objects.append(o)
    others.append(o)

In [11]:
oseeds = [( 70,205,12),
          (145,170,12),
          ( 50,160,12),
          (105,140,12),
          (190,220,12),
          (200,265,12),
          (220,335,20)]
for i in range(len(oseeds)):
    data.add_object_at( others[i], min_rs, max_rs, frame=0, seed=oseeds[i], segment_it=True )
    data.track(others[i], seed_frame=0, target_frames=range(1,len(data.images)), recenter_iterations=1)

Len: 1
Frame: 0
Added appearance for "o0" in frame 0 with seed coordinates [  70.  205.   12.]
      Optimum energy:  80145.0
      Nodes in/out:  7335 8025
Len: 1
Frame: 0
Added appearance for "o1" in frame 0 with seed coordinates [ 145.  170.   12.]
      Optimum energy:  79706.0
      Nodes in/out:  7451 7909
Len: 1
Frame: 0
Added appearance for "o2" in frame 0 with seed coordinates [  50.  160.   12.]
      Optimum energy:  85573.0
      Nodes in/out:  4837 10523
Len: 1
Frame: 0
Added appearance for "o3" in frame 0 with seed coordinates [ 105.  140.   12.]
      Optimum energy:  77165.0
      Nodes in/out:  6646 8714
Len: 1
Frame: 0
Added appearance for "o4" in frame 0 with seed coordinates [ 190.  220.   12.]
      Optimum energy:  84262.0
      Nodes in/out:  6537 8823
Len: 1
Frame: 0
Added appearance for "o5" in frame 0 with seed coordinates [ 200.  265.   12.]
      Optimum energy:  85762.0
      Nodes in/out:  7020 8340
Len: 1
Frame: 0
Added appearance for "o6" in frame 0 with

# Save / Load

In [8]:
data.save('/Users/prakash/Desktop/092216_PH-mCherry-02_w2SD mCherry.pkl')

In [9]:
data = Data4d( [], silent=False )
data.load('/Users/prakash/Desktop/092216_PH-mCherry-02_w2SD mCherry.pkl')
objects = range(len(data.object_names))
ecells = [0,1]
neighbors = range(2,2+12)
others = range(2+12,2+12+7)

Dimensions of frame 0 :  (31, 512, 512)
Working on "Ea" (object 1 of 2 )...
   Segmenting in frame 0...
      Optimum energy:  95372.0
      Nodes in/out:  7366 7994
      Volume:  40851.99999999993
Working on "Ep" (object 2 of 2 )...
   Segmenting in frame 0...
      Optimum energy:  88083.0
      Nodes in/out:  7008 8352
      Volume:  37384.166666666606


# Visualisation stuff 

In [12]:
w = data.show_frame(0,stackUnits=[1.,1.,5.],raise_window=True)
data.show_objects(ecells, show_surfaces=True, show_centers=True, colors=data.colors_gold)
#try:
#    data.show_objects(neighbors, show_surfaces=True, show_centers=True, colors=data.colors_green)
#    data.show_objects(others, show_surfaces=True, show_centers=True, colors=data.colors_blue)
#except:
#    None
#data.show_objects(neighbors, show_surfaces=True, show_centers=True, colors=data.colors_green)

In [11]:
for f in range(len(data.images)):
    print ('Exporting frame ',f+1,'of',len(data.images))
    w = data.show_frame(f,stackUnits=[1.,1.,5.],raise_window=False)
    data.show_objects(ecells, show_surfaces=True, show_centers=True, colors=data.colors_gold)
    try:
        data.show_objects(neighbors, show_surfaces=True, show_centers=True, colors=data.colors_green)
        data.show_objects(others, show_surfaces=True, show_centers=True, colors=data.colors_blue)
    except:
        None
    data.save_current_visualization( 'frame'+str(f)+'.jpg' )

INFO:spimagine.gui.glwidget | saving frame as frame0.jpg


Exporting frame  1 of 1


In [17]:
data.object_volumes[Ea]

[40851.99999999993]